In [ ]:
import numpy as np
import pandas as pd

In [ ]:
pd.set_option('display.max_columns',0)
pd.set_option('display.max_colwidth',0)

**Preprocessing match data**

In [ ]:
match_data = pd.read_csv('/content/drive/MyDrive/match.csv')

In [ ]:
match_data.head()

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9.0,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152.0,1.0,97.0,17982,7907451.0:4381761.0:31464.0:258649.0:4949790.0:3834305.0:3776849.0:6718340.0:3566240.0:4950364.0:7543647.0,18570,2653993.0:6718326.0:6718382.0:2486896.0:2288789.0:4950224.0:8187878.0:3083667.0:4950294.0:6718396.0:1585464.0,other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7.0,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117.0,2.0,74.0,33942,37351.0:46794.0:5406540.0:2231928.0:181404.0:1594319.0:7534687.0:7537067.0:1905847.0:8339701.0:7620346.0,33963,1506098.0:1749075.0:36665.0:2083409.0:7534652.0:5788320.0:1612610.0:5509524.0:5497274.0:3876613.0:8193310.0,other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4.0,Sy Ss,bat,Be Ct Gd,Brisbane,2021-01-02,night match,Bg Bh Le,2020/21,1476,GJ Dn,PJ Ge,165,8,126,171.0,6.0,119.0,33956,7869987.0:7620283.0:2076192.0:4002340.0:3063696.0:34061.0:3200756.0:4756982.0:7455818.0:49496.0:1506077.0,33921,7620269.0:2286437.0:87191.0:5786766.0:3114803.0:2535420.0:4967738.0:3995991.0:3890984.0:7353828.0:6818776.0:5419546.0,other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2.0,Oo,field,By Ol,Mount Maunganui,2021-01-02,night match,Sr Sh,2020/21,17681,GA St,TJ Pe,153,7,128,156.0,8.0,126.0,17982,7907451.0:4381761.0:31464.0:4949790.0:258649.0:3834305.0:3776849.0:7543647.0:6718340.0:3566240.0:7907458.0,18360,2319638.0:256080.0:7918280.0:3913447.0:2690498.0:6129276.0:2666705.0:6317142.0:3834375.0:2236086.0:4195827.0,other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21.0,Me Ss,field,Be Ol,Hobart,2021-01-02,day/night match,Bg Bh Le,2020/21,6348,GA Ad,NR Je,164,5,126,143.0,9.0,123.0,33928,4223883.0:2161599.0:1655436.0:5788418.0:319948.0:6249256.0:2340372.0:309056.0:3125849.0:6732004.0:6722540.0,33949,363047.0:2275097.0:3901078.0:2275195.0:4230127.0:2720759.0:2337117.0:6308098.0:7877232.0:3731307.0:4985546.0:2336473.0,other_domestic,33928,3125849.0


In [ ]:
# Null values
match_data.isnull().sum()

match id                  0  
team1                     0  
team2                     0  
winner                    0  
by                        0  
win amount                0  
toss winner               0  
toss decision             0  
venue                     0  
city                      178
match_dt                  0  
lighting                  0  
series_name               0  
season                    0  
ground_id                 0  
umpire1                   181
umpire2                   181
inning1_runs              0  
inning1_wickets           0  
inning1_balls             0  
inning2_runs              0  
inning2_wickets           0  
inning2_balls             0  
team1_id                  0  
team1_roster_ids          0  
team2_id                  0  
team2_roster_ids          0  
series_type               0  
winner_id                 0  
player_of_the_match_id    514
dtype: int64

In [ ]:
match_data.dropna(inplace=True)

In [ ]:
match_data['team1_roster_ids'] = match_data['team1_roster_ids'].str.split(':')
match_data['team2_roster_ids'] = match_data['team2_roster_ids'].str.split(':')

In [ ]:
# setting data types

match_data['team1_roster_ids'] = match_data['team1_roster_ids'].apply(lambda x: [int(round(float(val))) for val in x])
match_data['team2_roster_ids'] = match_data['team2_roster_ids'].apply(lambda x: [int(round(float(val))) for val in x])

match_data['win amount'] = [int(round(float(val))) for val in match_data['win amount']]

match_data['inning1_runs'] = [int(round(float(val))) for val in match_data['inning1_runs']]
match_data['inning1_wickets'] = [int(round(float(val))) for val in match_data['inning1_wickets']]
match_data['inning1_balls'] = [int(round(float(val))) for val in match_data['inning1_balls']]

match_data['inning2_runs'] = [int(round(float(val))) for val in match_data['inning2_runs']]
match_data['inning2_wickets'] = [int(round(float(val))) for val in match_data['inning2_wickets']]
match_data['inning2_balls'] = [int(round(float(val))) for val in match_data['inning2_balls']]

match_data['match_dt'] = pd.to_datetime(match_data['match_dt'], format='%Y-%m-%d')

In [ ]:
match_data.head()

,match id,team1,team2,winner,by,win amount,toss winner,toss decision,venue,city,match_dt,lighting,series_name,season,ground_id,umpire1,umpire2,inning1_runs,inning1_wickets,inning1_balls,inning2_runs,inning2_wickets,inning2_balls,team1_id,team1_roster_ids,team2_id,team2_roster_ids,series_type,winner_id,player_of_the_match_id
0,8638034,Nn Ds,Wn,Wn,wickets,9,Wn,field,By Ol,Mount Maunganui,2021-01-01,night match,Sr Sh,2020/21,17681,Dn Mw,TJ Pe,148,6,130,152,1,97,17982,"[7907451, 4381761, 31464, 258649, 4949790, 3834305, 3776849, 6718340, 3566240, 4950364, 7543647]",18570,"[2653993, 6718326, 6718382, 2486896, 2288789, 4950224, 8187878, 3083667, 4950294, 6718396, 1585464]",other_domestic,18570,NaN
1,8588005,Me Rs,Sy Tr,Sy Tr,runs,7,Sy Tr,field,Ca Ol,Carrara,2021-01-01,night match,Bg Bh Le,2020/21,6663,D Tr,PJ Ge,166,6,106,117,2,74,33942,"[37351, 46794, 5406540, 2231928, 181404, 1594319, 7534687, 7537067, 1905847, 8339701, 7620346]",33963,"[1506098, 1749075, 36665, 2083409, 7534652, 5788320, 1612610, 5509524, 5497274, 3876613, 8193310]",other_domestic,33963,1749075.0
2,8587837,Sy Ss,Be Ht,Be Ht,wickets,4,Sy Ss,bat,Be Ct Gd,Brisbane,2021-01-02,night match,Bg Bh Le,2020/21,1476,GJ Dn,PJ Ge,165,8,126,171,6,119,33956,"[7869987, 7620283, 2076192, 4002340, 3063696, 34061, 3200756, 4756982, 7455818, 49496, 1506077]",33921,"[7620269, 2286437, 87191, 5786766, 3114803, 2535420, 4967738, 3995991, 3890984, 7353828, 6818776, 5419546]",other_domestic,33921,3890984.0
3,8638041,Nn Ds,Oo,Oo,wickets,2,Oo,field,By Ol,Mount Maunganui,2021-01-02,night match,Sr Sh,2020/21,17681,GA St,TJ Pe,153,7,128,156,8,126,17982,"[7907451, 4381761, 31464, 4949790, 258649, 3834305, 3776849, 7543647, 6718340, 3566240, 7907458]",18360,"[2319638, 256080, 7918280, 3913447, 2690498, 6129276, 2666705, 6317142, 3834375, 2236086, 4195827]",other_domestic,18360,NaN
4,8587921,Ht Hs,Me Ss,Ht Hs,runs,21,Me Ss,field,Be Ol,Hobart,2021-01-02,day/night match,Bg Bh Le,2020/21,6348,GA Ad,NR Je,164,5,126,143,9,123,33928,"[4223883, 2161599, 1655436, 5788418, 319948, 6249256, 2340372, 309056, 3125849, 6732004, 6722540]",33949,"[363047, 2275097, 3901078, 2275195, 4230127, 2720759, 2337117, 6308098, 7877232, 3731307, 4985546, 2336473]",other_domestic,33928,3125849.0


**Preprocessing batsman data**

In [ ]:
batsman= pd.read_csv('/content/drive/MyDrive/batsman.csv')

In [ ]:
batsman.head()

,match id,batsman,batsman_id,batsman_details,is_batsman_captain,is_batsman_keeper,inning,runs,balls_faced,over_faced_first,wicket kind,out_by_bowler,out_by_fielder,bowler_id,bowler_details,is_bowler_keeper,is_bowler_captain,strike_rate,Fours,Sixes,match_dt
0,8638034,KD Ce,7907451.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,7,5,1.1,caught,JS Nm,TM Jn,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,140.00,1.0,NaN,2021-01-01
1,8638034,TL St,4381761.0,NZ:Right-hand bat:None:,0.0,1.0,1,46,46,1.2,caught,R Ra,MG Bl,6718382.0,NZ:Left-hand bat:Slow left-arm orthodox:,0.0,0.0,100.00,4.0,1.0,2021-01-01
2,8638034,HR Cr,4949790.0,NZ:Right-hand bat:Right-arm offbreak:,0.0,0.0,1,9,10,12.1,caught,PF Yd,R Ra,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,90.00,NaN,NaN,2021-01-01
3,8638034,BR Hn,3834305.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,28,22,13.6,caught,HK Bt,FH An,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,127.27,3.0,NaN,2021-01-01
4,8638034,SC Kn,3776849.0,NZ:Right-hand bat:Right-arm fast-medium:,0.0,0.0,1,18,13,17.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,138.46,NaN,1.0,2021-01-01


In [ ]:
# Null values
batsman.isnull().sum()

match id              0    
batsman               0    
batsman_id            0    
batsman_details       0    
is_batsman_captain    0    
is_batsman_keeper     0    
inning                0    
runs                  0    
balls_faced           0    
over_faced_first      0    
wicket kind           5328 
out_by_bowler         5328 
out_by_fielder        10645
bowler_id             6355 
bowler_details        6355 
is_bowler_keeper      5785 
is_bowler_captain     6355 
strike_rate           9    
Fours                 10283
Sixes                 15489
match_dt              0    
dtype: int64

In [ ]:
# Imputing missing strike rates with median
batsman['strike_rate'] = batsman['strike_rate'].fillna(batsman['strike_rate'].median())

In [ ]:
# Notice that the columns 'wicket kind' is NaN wherever the batsman was not out.
batsman['wicket kind'].fillna('not out', inplace=True)

In [ ]:
batsman['Fours'].fillna(0, inplace=True)
batsman['Sixes'].fillna(0, inplace=True)
batsman['bowler_id'].fillna(0, inplace=True)

In [ ]:
# Extracting batting style from batsman details

batsman['batsman_details'] = batsman['batsman_details'].apply(lambda x: x.split(':'))
batsman['batsman_style'] = batsman['batsman_details'].apply(lambda x: x[1])
batsman.drop(['batsman_details'],axis=1, inplace=True)

In [ ]:
# Extracting bowling style from bowler details

batsman['bowler_details'] = batsman['bowler_details'].astype(str).apply(lambda x: x.split(':'))
batsman['bowler_style'] = batsman['bowler_details'].apply(lambda x: x[2] if len(x) > 2 else 'none') # Handle cases with less than 3 elements
batsman.drop(['bowler_details'],axis=1, inplace=True)

In [ ]:
# Setting data types

batsman['batsman_id'] = [round(val) for val in batsman['batsman_id']]
batsman['match_dt'] = pd.to_datetime(batsman['match_dt'], format='%Y-%m-%d')

In [ ]:
batsman.drop(['is_batsman_captain', 'is_batsman_keeper', 'over_faced_first', 'is_bowler_keeper', 'is_bowler_captain'], axis=1, inplace=True)

In [ ]:
batsman.head()

,match id,batsman,batsman_id,inning,runs,balls_faced,wicket kind,out_by_bowler,out_by_fielder,bowler_id,strike_rate,Fours,Sixes,match_dt,batsman_style,bowler_style
0,8638034,KD Ce,7907451,1,7,5,caught,JS Nm,TM Jn,2486896.0,140.00,1.0,0.0,2021-01-01,Right-hand bat,Right-arm medium-fast
1,8638034,TL St,4381761,1,46,46,caught,R Ra,MG Bl,6718382.0,100.00,4.0,1.0,2021-01-01,Right-hand bat,Slow left-arm orthodox
2,8638034,HR Cr,4949790,1,9,10,caught,PF Yd,R Ra,4950294.0,90.00,0.0,0.0,2021-01-01,Right-hand bat,Legbreak googly
3,8638034,BR Hn,3834305,1,28,22,caught,HK Bt,FH An,1585464.0,127.27,3.0,0.0,2021-01-01,Right-hand bat,Right-arm medium-fast
4,8638034,SC Kn,3776849,1,18,13,not out,NaN,NaN,0.0,138.46,0.0,1.0,2021-01-01,Right-hand bat,none


**Preprocessing bowler data**

In [ ]:
bowler= pd.read_csv('/content/drive/MyDrive/bowler.csv')

In [ ]:
bowler.head()

,match id,bowler,bowler_id,bowler_details,is_bowler_captain,is_bowler_keeper,inning,runs,wicket_count,balls_bowled,economy,maiden,dots,Fours,Sixes,wides,noballs,match_dt
0,8638034,BV Ss,6718396.0,NZ:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,21,0,12,10.50,0,3.0,1,1,3,1,2021-01-01
1,8638034,HK Bt,1585464.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,34,1,24,8.50,0,7.0,3,1,2,0,2021-01-01
2,8638034,JS Nm,2486896.0,NZ:Left-hand bat:Right-arm medium-fast:,0.0,0.0,1,23,2,24,5.75,0,12.0,1,1,0,0,2021-01-01
3,8638034,LV vn Bk,3083667.0,NED:Right-hand bat:Right-arm medium-fast:,0.0,0.0,1,31,1,24,7.75,0,9.0,4,0,2,0,2021-01-01
4,8638034,PF Yd,4950294.0,NZ:Right-hand bat:Legbreak googly:,0.0,0.0,1,21,1,24,5.25,0,8.0,0,0,1,0,2021-01-01


In [ ]:
# Null values
bowler.isnull().sum()

match id             0
bowler               0
bowler_id            0
bowler_details       0
is_bowler_captain    0
is_bowler_keeper     0
inning               0
runs                 0
wicket_count         0
balls_bowled         0
economy              0
maiden               0
dots                 0
Fours                0
Sixes                0
wides                0
noballs              0
match_dt             0
dtype: int64

In [ ]:
# Extracting bowling style from bowler details

bowler['bowler_details'] = bowler['bowler_details'].apply(lambda x: x.split(':'))
bowler['bowler_style'] = bowler['bowler_details'].apply(lambda x: x[2] if len(x) > 2 else 'none') # Handle cases with less than 3 elements
bowler.drop(['bowler_details'],axis=1, inplace=True)

In [ ]:
# Setting data types

bowler['bowler_id'] = [round(val) for val in bowler['bowler_id']]
bowler['dots'] = [round(val) for val in bowler['dots']]
bowler['match_dt'] = pd.to_datetime(bowler['match_dt'], format='%Y-%m-%d')
bowler['maiden'] = [round(val) for val in bowler['maiden']]
bowler['match_dt'] = pd.to_datetime(bowler['match_dt'], format='%Y-%m-%d')

In [ ]:
bowler.drop(['is_bowler_captain', 'is_bowler_keeper'], axis=1, inplace=True)

In [ ]:
match_data.to_csv('match_data.csv', index=False)
batsman.to_csv('batsman.csv', index=False)
bowler.to_csv('bowler.csv', index=False)